# Event-based Simulation with SimPy

In [2]:
import simpy as sp
import numpy as np
import scipy.stats

## SimPy Basics

SimPy has a few basic concepts to let you perform event-based simulation:
1. **Environment:** think of this as "the world" in which your simulation occurs
2. **Event:** an event that occurs in the environment -- it can be a happening, from time ticking to an action occurring
3. **Process:** described by Python generator functions -- they create events and `yield` (yeet) them out to the world
4. **Timeout:** a special time-based event, that describes how long it takes for an event to occur (assumed units)
5. **Resource:** an object representing a resource that can be acquired or released by Processes (gas station with limited fuel pumps)
6. **Container:** an object representing a resource that can be produced or consumed (like electrical power or apples)
7. **Store:** an object representing a store that can replenish or sell its items (dynamic quantities)

One thing that is key about event-based simulation is that, much like embedded systems, all processes are **interruptible** by priority, meaning that processes can be at different priorities (just like real life).

If you need to yield multiple events (actions) at a time, use the `&` operator.  If you need to yield at least one event (whichever event is closest), use the `|` operator.

### DMV customer example

Just to show everything in context, I'll modify the [Bank example](https://simpy.readthedocs.io/en/latest/examples/bank_renege.html) to the DMV:

In [13]:
RANDOM_SEED = 42
NEW_CUSTOMERS = 10  # Total number of customers
INTERVAL_CUSTOMERS = 10.0  # Generate new customers roughly every x seconds
MIN_PATIENCE = 10  # Min. customer patience
MAX_PATIENCE = 40  # Max. customer patience


def source(env, number, interval, counter, the_rng):
    """Source generates customers randomly"""
    for i in range(number):
        c = customer(env, 'Customer%02d' % i, counter, 12.0, the_rng)
        env.process(c)
        t = the_rng.exponential(interval)
        yield env.timeout(t)


def customer(env, name, counter, time_in_bank, the_rng):
    """Customer arrives, is served and leaves."""
    arrive = env.now
    print('at time %7.4f, %s arrived' % (arrive, name))

    with counter.request() as req:
        patience = the_rng.uniform(MIN_PATIENCE, MAX_PATIENCE)
        # Wait for the counter or abort at the end of our tether
        results = yield req | env.timeout(patience)

        wait = env.now - arrive

        if req in results:
            # We got to the counter
            print('at time %7.4f, %s waited %6.3f' % (env.now, name, wait))

            tib = the_rng.exponential(time_in_bank)
            yield env.timeout(tib)
            print('at time %7.4f, %s finished their transactions' % (env.now, name))

        else:
            # We reneged
            print('at time %7.4f, %s left the line after waiting %6.3f minutes' % (env.now, name, wait))


# Setup and start the simulation
print('SIMULATION: DMV in all of its glory (time units are minutes)')
the_rng = np.random.default_rng(RANDOM_SEED)
env = sp.Environment()

# Start processes and run
counter = sp.Resource(env, capacity=1)
env.process(source(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter, the_rng))
env.run()

SIMULATION: DMV in all of its glory (time units are minutes)
at time  0.0000, Customer00 arrived
at time  0.0000, Customer00 waited  0.000
at time 24.0421, Customer01 arrived
at time 26.8400, Customer02 arrived
at time 28.6171, Customer00 finished their transactions
at time 28.6171, Customer01 waited  4.575
at time 41.3666, Customer03 arrived
at time 42.1596, Customer04 arrived
at time 42.8639, Customer05 arrived
at time 59.6742, Customer02 left the line after waiting 32.834 minutes
at time 60.1772, Customer06 arrived
at time 64.8782, Customer03 left the line after waiting 23.512 minutes
at time 66.1087, Customer01 finished their transactions
at time 66.1087, Customer04 waited 23.949
at time 67.2076, Customer04 finished their transactions
at time 67.2076, Customer05 waited 24.344
at time 70.9898, Customer05 finished their transactions
at time 70.9898, Customer06 waited 10.813
at time 72.4931, Customer07 arrived
at time 76.6229, Customer08 arrived
at time 78.8587, Customer09 arrived
at 

### Sample SimPy problem

Let's perform a simulation of four electric cars (EV, electric vehicles) trying to drive to and use a battery charging station. A car will perform three actions:
1. Driving to the EV charging station (accept the driving time as an input parameter).
2. Request/acquire a charging spot.
3. Charge the battery (accept the time it takes to charge the battery as an input parameter).

Define the charging station as only having two spots, so we have a likely chance of at least one car waiting for charging to complete.
Let's use a uniform distribution between 2 and 6 miles to model the distance each car has to drive in order to reach the EV charging station. Let's also assume that the cars can drive 35 miles per hour to reach the charging station.  Let's also use a uniform distribution to model the number of minutes that each car has to charge, from 30 minutes to 90 minutes.

LPT: Don't forget the random seed!

In [22]:
DISTANCE_MIN = 2  # miles
DISTANCE_MAX = 6  # miles
NUM_OF_CARS = 8  # cars
DRIVING_SPEED = 35  # miles per hour
CHARGING_MIN = 30  # mins
CHARGING_MAX = 90  # mins
CHARGING_SPOTS = 3
RANDOM_SEED = 7


def create_cars(env: sp.Environment, charging_station: sp.Resource,
                rng: np.random.Generator,
                number_of_cars: int, driving_distance_min: float,
                driving_distance_max: float, driving_speed: float,
                car_charging_min: float, car_charging_max: float):
    for car_id in range(number_of_cars):
        charging_time = rng.uniform(car_charging_min, car_charging_max)
        driving_distance = rng.uniform(driving_distance_min, driving_distance_max)
        driving_time = 60 * driving_distance / driving_speed  # convert hours to minutes
        env.process(electric_car(env, charging_station, car_id, driving_time, charging_time))
    print(f"At {env.now}, all cars have been added to the environment.")


def electric_car(env: sp.Environment, charging_station: sp.Resource,
                 name: str, driving_time: float, charging_time: float):
    print(f"At {env.now:.2f}, Car {name} with t_drive {driving_time:.2f} and t_charge {charging_time:.2f} has spawned.")
    print(f"At {env.now:.2f}, Car {name} is driving to the EV charging station.")
    yield env.timeout(driving_time)  # 1. driving to the station
    
    with charging_station.request() as charging_spot:  # 2a. request/acquire a charging spot
        wait_start = env.now
        print(f"At {env.now:.2f}, Car {name} is waiting for an EV charging spot.")
        yield charging_spot  # 2b. wait for an available EV charging spot
        
        wait_completed = env.now
        wait_duration = wait_completed - wait_start
        print(f"At {env.now:.2f}, Car {name} has acquired an EV charging spot after waiting {wait_duration:.2f}.")
        yield env.timeout(charging_time)  # 3. charge the battery
        
    print(f"At {env.now:.2f}, Car {name} has finished charging and has left.")

In [23]:
the_rng = np.random.default_rng(RANDOM_SEED)
the_env = sp.Environment()
the_charging_station = sp.Resource(the_env, capacity=CHARGING_SPOTS)
create_cars(the_env, the_charging_station, the_rng, NUM_OF_CARS,
            DISTANCE_MIN, DISTANCE_MAX, DRIVING_SPEED,
            CHARGING_MIN, CHARGING_MAX)
the_env.run()

At 0, all cars have been added to the environment.
At 0.00, Car 0 with t_drive 9.58 and t_charge 67.51 has spawned.
At 0.00, Car 0 is driving to the EV charging station.
At 0.00, Car 1 with t_drive 4.97 and t_charge 76.54 has spawned.
At 0.00, Car 1 is driving to the EV charging station.
At 0.00, Car 2 with t_drive 9.42 and t_charge 48.01 has spawned.
At 0.00, Car 2 is driving to the EV charging station.
At 0.00, Car 3 with t_drive 9.06 and t_charge 30.32 has spawned.
At 0.00, Car 3 is driving to the EV charging station.
At 0.00, Car 4 with t_drive 6.64 and t_charge 77.82 has spawned.
At 0.00, Car 4 is driving to the EV charging station.
At 0.00, Car 5 with t_drive 5.34 and t_charge 48.18 has spawned.
At 0.00, Car 5 is driving to the EV charging station.
At 0.00, Car 6 with t_drive 6.48 and t_charge 45.29 has spawned.
At 0.00, Car 6 is driving to the EV charging station.
At 0.00, Car 7 with t_drive 7.22 and t_charge 60.27 has spawned.
At 0.00, Car 7 is driving to the EV charging statio

## Handling priorities and interrupts

Let's bump the EV charging simulation up now, to 10 cars, and pretend that a couple of the cars have made reservations online, so they can jump the queue (`simpy.PriorityResource`).  If the class wants, we could even say someone has a VIP card that can kick someone off of a pump (`simpy.PreemptiveResource`).

We could even use interrupts in a non-mean way, to indicate when a driver might have just gotten a text message and needs to leave and be somewhere instead.  Let's discuss how to model this event!

## Using a `class` to define more complex items that have multiple simultaneous processes

In the initialization, make sure to do two things:
1. Create a `<classname>_reactivate` member that is a generic `Event` that can be used to trigger interrupts.
2. Define each member function as having a `process` for each process that the complex item does.

Let's make our EV a bit fancier!  We can add the following:
1. We now keep track of our battery level.
2. We have a solar charger on our EV that automatically charges once we drive, and disengages on interrupt (if we drive).

Let's make a simulation where our driver will drive to the mall, shop for some random time, then come back and drive away (which should interrupt the solar charging process).

## Monitoring your SimPy simulation

There are three ways to go about monitoring a simulation, in order from easy to difficult:
1. Implement your own monitoring harness (basically, add input/output parameters for monitoring into your process functions)
2. Patching (creating a wrapper) for Simpy `Events` ([see this link](https://simpy.readthedocs.io/en/latest/topical_guides/monitoring.html#event-tracing))
3. Patching (creating a wrapper) for SimPy `Resources` ([see this link](https://simpy.readthedocs.io/en/latest/topical_guides/monitoring.html#resource-usage))

# Using SimPy for real-time ECE problems

Turns out that SimPy also has a RealtimeEnvironment, which lets you synchronize events with wall-clock time.

This means that we can perform the following simulations:
1. Hardware-in-the-loop testing (VERY useful in electronics, embedded systems, and medical devices)
2. Testing that requires human interaction (energy, sustainability, etc.)

**Let's discuss:** what might be an event-based simulation problem in your specialization